<a href="https://colab.research.google.com/github/gtbook/robotics/blob/main/S71_drone_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q -U gtbook

Note: you may need to restart the kernel to use updated packages.


In [2]:
import math
import numpy as np
import gtsam 

# Moving in Three Dimensions

> Motion in 3D has 6 DOF.

<img src="Figures7/S71-Autonomous_camera_drone-03.jpg" alt="Splash image with steampunk drones in various orientations" width="40%" align=center style="vertical-align:middle;margin:10px 0px">

In Section 6.1 we introduced the space of 2D rigid transformations $SE(2)$, and showed how matrices can be used to represent both
rotation and translation in two dimensions.
At that time, we promised that this could be easily generalized to the case of rotation and
translation in three dimensions.  In this section, we deliver on that promise, and introduce two new spaces:
the 3D rotations $SO(3)$ and 3D rigid transformations $SE(3)$.

In addition to representing the *pose* of a drone in 3D we are also interested in
representing velocity.
This includes the both the linear velocity of the drone, which corresponds to the velocity
of the origin of the body-attached frame, as well as the rate of change in the orientation of the
body-attached frame.

## Rotations in 3D aka SO(3)

In Section 6.1, we constructed a rotation matrix $R^0_1 \in SO(2)$
by projecting the axes of Frame 1 onto Frame 0.
The extension to 3D is indeed straightforward.
Again, we merely project the axes of Frame 1 onto Frame 0,
but in the 3D case, each frame is equipped with an additional $z$-axis:

$$
R_{1}^{0}
=\begin{bmatrix}
x_1 \cdot x_0 & y_1 \cdot x_0 & z_1 \cdot x_0 \\
x_1 \cdot y_0 & y_1 \cdot y_0 & z_1 \cdot y_0 \\
x_1 \cdot z_0 & y_1 \cdot z_0 & z_1 \cdot z_0 \\
\end{bmatrix}
$$

The expression for rotational coordinate transformations is now exactly the
same as in Chapter 6.
Given a point $p$ with coordinates expressed relative to the body-attached
frame B, we compute its coordinates $p^s$ in the "spatial" frame $S$:

$$
p^s = R^s_b p^b
$$

Extending the semantics from 2D rotations in $SO(2)$ to 3D rotations represented by $SO(3)$,
the columns
of $R_{b}^{s}$ represent the axes of frame $B$ in the $S$ coordinate
frame: 

$$
R_{b}^{s}=\left[\begin{array}{ccc}
\hat{x}_{b}^{s} & \hat{y}_{b}^{s} & \hat{z}_{b}^{s}\end{array}\right]
$$

The 3D rotations together with composition constitute the **special
orthogonal group** $SO(3)$, hence the abbreviation "SO". It is made up of all $3\times3$ orthonormal
matrices with determinant $+1$. This group has matrix multiplication as its
composition operation, 
but unlike in 2D  *3D rotations do not commute*, i.e., in general $R_{2}R_{1}\neq R_{1}R_{2}$.
The group so obtained it is called "special" because of the +1 determinant. Orthonormal matrices have either a -1 or +1 determinant, but those with -1 are *reflections*, which we specifically exclude here.

It is often useful to refer to rotations about one of the coordinate axes.
These basic rotation matrices are given by

$$
R_{x,\phi}
=\begin{bmatrix}
1 & 0 & 0 \\
0 & \cos \phi & - \sin \phi \\
0 & \sin \phi & \cos \phi 
\end{bmatrix}
~~~
R_{y,\theta}
=\begin{bmatrix}
\cos \theta &      0   & \sin \theta \\
0            &     1   &  0 \\
-\sin \theta & 0       & \cos \theta
\end{bmatrix}
~~~
R_{z,\psi}
=\begin{bmatrix}
\cos \psi &-\sin \psi & 0 \\
\sin \psi &\cos \psi & 0 \\
0 & 0   &  1
\end{bmatrix}
$$

Note, in particular, the form of $R_{z,\psi}$, the rotation about the $z$-axis.
The upper-left $2\times 2$ block of this matrix is exactly a rotation matrix $R \in SO(2)$
as introduced in Section 6.1.
This can be understood by realizing that rotation in the $xy$ plane is actually
a rotation about an implicit $z$- axis that points "out of the page."

## 3D Rigid transforms aka SE(3)

In a similar way we can extend the group of 2D rigid transforms $SE(2)$ to the case
of rotation and translation in 3D.
Suppose a point $p$ is rigidly attached to frame $B$, and
we wish to determine its coordinates with respect to the reference frame $S$.
If the coordinates of $p$ with respect to frame $B$ are given by $p^b$,
then we can express it in frame $S$ as follows:

$$
p^{s}=R_{b}^{s}p^{b}+t_{b}^{s}
$$ 

Above $R^s_b \in SO(3)$ is the rotation matrix that specifies the orientation
of frame $B$ w.r.t. frame $S$,
and  $t^s_b \in \mathbb{R}^3$ gives the location of the origin
of frame $B$ w.r.t. frame $S$ (specified in the coordinates of frame $S$).


We denote this transform by
$T_{b}^{s}\doteq\left(R_{b}^{s},\,t_{b}^{s}\right)$. These transforms $T_{b}^{s}$ are elements of
the  **special Euclidean group** $SE(3)$. Similarly to the previous chapter we can view the 
group $SE(3)$ as a subgroup of the general linear group $GL(4)$ of degree 4, 
by embedding the rotation and translation into a $4\times4$ invertible matrix defined as

$$
T_{b}^{s}=\begin{bmatrix}
R_{b}^{s} & t_{b}^{s}\\
0 & 1
\end{bmatrix}.
$$

We can then implement the group operation as simple matrix multiplication.

And, in analogy with the 2D case, by embedding 3D points in a four-vector, a
3D rigid transform acting on a point can be implemented by matrix-vector
multiplication:

$$
\begin{bmatrix}
R_{b}^{s} & t_{b}^{s}\\
0 & 1
\end{bmatrix}
\begin{bmatrix}
p^{b}\\
1
\end{bmatrix}
=\begin{bmatrix}
R_{b}^{s}p^{b}+t_{b}^{s}\\
1
\end{bmatrix}
$$

## The Group $SE(3)$ in GTSAM

> `Pose3` is a non-commutative group which acts on `Point3`.

$SE(3)$ is a group, as the following properties all hold:

1. **Closure**: For all transforms $T, T' \in SE(3)$, their product is also in $SE(3)$, i.e., $T T' \in SE(3)$.
2. **Identity Element**: The $4\times 4$ identity matrix $I$ is included in the group, and for
all $T \in SE(3)$ we have $T I = I T = T$.
3. **Inverse**: For every $T \in SE(3)$ there exists $T^{-1} \in SE(3)$ such that $T^{-1}T = T T^{-1} = I$.
4. **Associativity**: For all $T_1, T_2, T_3 \in SE(3)$, $(T_1 T_2) T_3 = T_1 (T_2 T_3)$.

3D rigid transforms do *not* commute. The inverse $T^{-1}$ is given by:

$$
T^{-1} = \begin{bmatrix}R & d\\0_2 & 1\end{bmatrix}^{-1}
 = \begin{bmatrix}R^T & -R^T d\\0_2 & 1\end{bmatrix}
$$

Again, all of this is built into GTSAM, where both 3D poses and 3D rigid transforms are represented by the type `Pose3`, with rotation matrices represented by `Rot3`:

In [4]:
R = gtsam.Rot3.Yaw(math.degrees(20)) # rotation around Z-axis by 20 degrees
T = gtsam.Pose3(R, [1,2,3])
print(f"3D Pose {T} corresponding to transformation matrix:\n{T.matrix()}")

3D Pose R: [
	-0.720878, -0.693062, 0;
	0.693062, -0.720878, 0;
	0, 0, 1
]
t: 1 2 3
 corresponding to transformation matrix:
[[-0.72087779 -0.6930622   0.          1.        ]
 [ 0.6930622  -0.72087779  0.          2.        ]
 [ 0.          0.          1.          3.        ]
 [ 0.          0.          0.          1.        ]]


3D transforms form a *non-commutative* group, as demonstrated here in code:

In [5]:
print(isinstance(T * T, gtsam.Pose3)) # closure
I = gtsam.Pose3.Identity()
print(T.equals(T * I, 1e-9)) # identity
print((T * T.inverse()).equals(I, 1e-9)) # inverse
T1, T2, T3 = T, gtsam.Pose3(gtsam.Rot3.Roll(0.1), [1,2,3]), gtsam.Pose3(gtsam.Rot3.Roll(0.2), [1,2,3])
print(((T1 * T2)* T3).equals(T1 * (T2 * T3), 1e-9)) # associative
print((T1 * T2).equals(T2 * T1, 1e-9)) # NOT commutative

True
True
True
True
False


Finally, 3D transforms can act on 3D points, which we can do using matrix multiplication, or using the `Pose3.transformFrom` method:

In [6]:
P1 = gtsam.Point3(2, 4, 3)
print(f"P0 = {T.matrix() @ [2, 4, 3, 1]}")  # need to make P0 homogeneous
print(f"P0 = {T.transformFrom(P1)}")

P0 = [-3.21400437  0.50261322  6.          1.        ]
P0 = [-3.21400437  0.50261322  6.        ]


You can see from the code snippet above that both approaches yield the same numerical values for the first three coordinates.

## Angular Velocity

The linear velocity $v$ of the origin of a moving frame is easy to understand.
This velocity is obtained merely by computing the derivative of the coordinate vector for the frame's origin.
The derivative of orientation is more interesting.

We begin by introducing the family of $3 \times 3$ skew symmetric matrices, denoted by $\mathfrak{so}(3)$. 
The relationship between $\mathfrak{so}(3)$ and $SO(3)$ will become apparent shortly.
A matrix $S$ is said to be a **skew symmetric matrix** if

$$
S + S^T = 0
$$

If we examine this equation for each element of $S$, we obtain nine equations, 

$$ s_{i,j} = - s_{j,i}, ~~~~ i,j = 1,2,3 $$

When $i=j$, we have $s_{ii} = -s_{ii}$, which is satisfied only if $s_{ii} = 0$.
Thus, the diagonal elements of a skew symmetric matrix are all zeros.
The remaining equations constrain the off-diagonal entries, such that
every $S \in so(3)$ can be written as

$$
\hat{s} = S =
\begin{bmatrix}
0 & -s_z &s_y \\
s_z & 0 & -s_x \\
-s_y & s_x & 0
\end{bmatrix}
$$

Note that we define the **hat operator** $\hat{\cdot}$, which maps a vector $s = (s_x, s_y, s_z)$ to
a skew symmetric matrix as above.

Skew symmetric matrices play a key role in representing the angular rate of change of coordinate frames.
Let $R(t)$ be a time-varying rotation matrix.
Because $R(t) \in SO(3)$, at each time $t$ we have $R^T(t) R(t) = I$.
If we differentiate both sides of this equation, we obtain

$$
\begin{aligned}
\frac{d}{dt} \left[ R^T(t) R(t) \right] &= \frac{d}{dt} I  \\
     \dot{R}^T R + R^T \dot{R} &= 0
\end{aligned}
$$

In other words, $R^T \dot{R} = - \dot{R}^T R$ and hence the matrix $R^T \dot{R}$ *is a skew symmetric matrix*

$$R^T \dot{R} = \hat{\omega}$$ 

with $\omega$ some three-vector. Multiplying by $R$ on both sides immediately gives an equation of $\dot{R}$ in terms of $\omega$:

$$\dot{R} = R \hat{\omega}.$$

The above gives an expression for the derivative of a rotation matrix $\dot{R}$ that
will be used below in our formulation of drone dynamics.
The vector $\omega$ is called the **angular velocity**,
and is the same angular velocity that you may have
learned in an introductory physics course.